In [1]:
# --- Cell 1: Cài đặt thư viện ---
!pip install requests beautifulsoup4 tqdm

# --- Cell 2: Import thư viện ---
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

BASE_URL = "https://dermnetnz.org"

# --- Cell 3: Lấy tất cả category, kể cả bị ẩn ---
def get_categories():
    url = f"{BASE_URL}/topics"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    categories = []
    for a in soup.select('a.js-topic-category'):
        name = a.get('data-category', '').strip()
        if name and name not in categories:
            categories.append(name)
    return categories

# --- Cell 4: Hàm lấy danh sách topic theo category ---
def get_topics_by_category(category):
    url = f"{BASE_URL}/topics"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    # Tìm thẻ category
    cat_tag = None
    for a in soup.select('a.js-topic-category'):
        if a['data-category'].strip() == category:
            cat_tag = a
            break
    if not cat_tag:
        return []
    # Lấy vị trí category
    pos = cat_tag['data-pos']
    # Tìm block topic tương ứng
    topic_block = soup.select_one(f'div[data-category="{category}"]')
    if not topic_block:
        # fallback: lấy tất cả topic
        topic_block = soup
    topics = []
    for a in topic_block.select('a[href^="/topics/"]'):
        href = a['href']
        if href.startswith('/topics/') and len(href) > 8:
            topics.append((a.text.strip(), BASE_URL + href))
    return topics

# --- Cell 5: Lấy tất cả Q&A trong topic ---
def scrape_qa_from_topic(topic_url):
    r = requests.get(topic_url)
    soup = BeautifulSoup(r.text, "html.parser")
    qa_list = []
    for block in soup.select('.text-block__wrap'):
        # Tìm tất cả h2 trong block
        for q in block.find_all('h2'):
            question = q.get_text(strip=True)
            # Lấy tất cả text sau h2 cho đến thẻ h2 tiếp theo hoặc hết block
            answer_parts = []
            for sib in q.find_next_siblings():
                if sib.name == 'h2':
                    break
                answer_parts.append(sib.get_text(separator=' ', strip=True))
            answer = ' '.join(answer_parts).strip()
            if question and answer:
                qa_list.append({'question': question, 'answer': answer})
    return qa_list

# --- Cell 6: Hàm lưu dữ liệu ra file ---
def safe_filename(name):
    # Loại bỏ các ký tự không hợp lệ cho tên file/thư mục trên mọi hệ điều hành
    return re.sub(r'[\\/*?:"<>|\'`]', '', name).strip().replace(' ', '_')

def save_qa(category, topic, qa_list, base_folder="dermnetnz_qa"):
    cat_dir = os.path.join(base_folder, safe_filename(category))
    os.makedirs(cat_dir, exist_ok=True)
    topic_file = os.path.join(cat_dir, safe_filename(topic) + ".txt")
    with open(topic_file, "w", encoding="utf-8") as f:
        for qa in qa_list:
            f.write("Q: " + qa['question'] + "\n")
            f.write("A: " + qa['answer'] + "\n\n")

# --- Cell 7: Chạy toàn bộ pipeline ---
categories = get_categories()
for category in tqdm(categories, desc="Categories"):
    topics = get_topics_by_category(category)
    for topic_name, topic_url in tqdm(topics, desc=f"Topics in {category}", leave=False):
        qa_list = scrape_qa_from_topic(topic_url)
        if qa_list:
            save_qa(category, topic_name, qa_list)

# --- Cell 8: Chạy toàn bộ pipeline và log ---
categories = get_categories()
print(f"Tổng số category: {len(categories)}\n")

for category in tqdm(categories, desc="Categories"):
    topics = get_topics_by_category(category)
    print(f"\nCategory: {category} - Số topic: {len(topics)}")
    for topic_name, topic_url in tqdm(topics, desc=f"  Topics in {category}", leave=False):
        qa_list = scrape_qa_from_topic(topic_url)
        print(f"    Topic: {topic_name} - Số cặp Q&A: {len(qa_list)}")
        if qa_list:
            save_qa(category, topic_name, qa_list)


# Sau khi chạy xong, dữ liệu sẽ nằm trong thư mục dermnetnz_qa, mỗi category là 1 folder, mỗi topic là 1 file txt chứa các cặp Q/A.

Categories: 100%|██████████| 23/23 [08:26<00:00, 22.02s/it]


Tổng số category: 23



Categories:   0%|          | 0/23 [00:00<?, ?it/s]


Category: Reactions - Số topic: 239



  Topics in Reactions:   0%|          | 0/239 [00:00<?, ?it/s]

...

    Topic: Wound infections - Số cặp Q&A: 5
    Topic: What is dermatology - Số cặp Q&A: 5
    Topic: Wound cleansers - Số cặp Q&A: 4



  Topics in Terminology:  99%|█████████▊| 236/239 [00:20<00:00, 12.06it/s]

    Topic: Wells syndrome - Số cặp Q&A: 9
    Topic: Warfarin induced skin necrosis - Số cặp Q&A: 7
    Topic: Wound closure - Số cặp Q&A: 4



Categories: 100%|██████████| 23/23 [08:11<00:00, 21.39s/it]

    Topic: Winter itch - Số cặp Q&A: 7
    Topic: Wells syndrome pathology - Số cặp Q&A: 3
